# 标准随机基准测试

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

**标准随机基准测试（Standard Randomized Benchmarking, SRB）**，是一项被大家所熟知的用于刻画量子设备性能的技术。通过运行并测量一系列由不同深度的 Clifford 门组成的随机量子电路，来衡量门的平均性能。该教程展示如何在[百度量子平台](https://quantum.baidu.com/)上使用 **SRB** 来刻画衡量量子设备的性能，并计算 **Clifford 门错误率（EPC）**。

## 理论

### Clifford 门
Clifford 群作用在 Pauli 群（$P_n$）上，使其演化后仍然为自身，即：

$$
Cl(P_n)\equiv \left\{U\vert UP_nU^{\dagger}=P_n\right\}.
$$

这就是 $P_n$ 对应的 Clifford 群， 也称作 $P_n$ 的 "normalizer"。而由 Clifford 群元组成的量子电路就称为 Clifford 电路，或称为 stabilizer 线路。上述所说的性质，使得 Clifford 电路可以被经典计算机在多项式复杂度内有效模拟 (Gottesman-Knill 定理)[1]，因此可以适用于大型量子设备的基准测试，同时 Clifford 电路还广泛应用在量子纠错理论中。

Clifford 门可以由一组基本的量子门生成，即 {H, S, CZ} （也有 {H, S, T, CNOT} 等，本质上相同）。量噪中实现了任意 $n$ 量子比特的 Clifford 门方法，可以通过以下代码调用

In [ ]:
from qcompute_qep.quantum.clifford import Clifford,random_clifford
from qcompute_qep.benchmarking.standardrb import StandardRB

In [ ]:
n = 2 # number of qubits
cliff = Clifford(n)
print(cliff) # print the Clifford Gate in elementary gate form


上面的结果即为 $2$ 量子比特的 Clifford Gate，我们所采用的构造方法来自该论文[2]。

### 随机电路结构

SRB 中常见的随机 Clifford 电路结构如下图所示

![random-clifford-circuit](figures/srb-clifford-circuit.png "图1：随机 Clifford 电路。")

图中我们先加入 $m$ 个随机的 Clifford 门，之后再加入一个逆门 $C_{m+1}$，表示为所有 $m$ 个 Clifford 门的逆作用，即：

$$
C_{m+1} = \left(C_1C_2\cdots C_{m-1}C_{m}\right)^{-1}.
$$

则该线路在理想情况下输出仍然为初始态：

$$
C_{m+1}C_{m}C_{m-1}\cdots C_2C_1\vert \phi \rangle = \vert \phi \rangle.
$$

若初始态为 $\vert 0 \rangle$，则在理想电路的情况下无论多少次制备测量，结果都为 $\vert 0 \rangle$。

### 随机基准测试

当量子设备（主要是量子门）存在噪声时，结果会不同。随着量子门的数目增加（即电路深度的增加），噪声或误差会不断累积放大。我们可以直觉地判断，如果一个量子设备所提供的量子门性能好，误差小，即使门的数量增加累计的误差也比较小。所以我们通过搭建并测量一系列不同深度的 Clifford 电路，将数据代入方程

$$
p(m) = Af^m+B,
$$

进行拟合得到结果，以此判断量子设备的性能。在这个公式中，$m$ 为深度，$p(m)$ 为 $\textrm{Tr}\left[OC_{m+1}C_{m}C_{m-1}\cdots C_2C_1\vert \phi \rangle\!\langle\phi\vert\right]$，即在 $m$ 深度电路下的测量结果，$f$为保真度，来衡量 Clifford 门的平均性能，$A$ 和 $B$ 用来吸收态制备与测量误差 （SPAM）。下面我们通过案例来演示说明。

## 实践

### 单量子比特电路

单量子比特的电路如图所示

![single-qubit](figures/srb-single-clifford-circuit.png "图2：单量子比特电路。")

下面代码中我们在每个深度下都构造 $20$ 个随机电路，测量并取平均值。运行代码，并查看运行展示的最终结果

In [ ]:
import QCompute

# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
QCompute.Define.hubToken = "Token"
qc = QCompute.BackendName.CloudBaiduQPUQian

single_qubits = [0]
rb = StandardRB()
single_qubit_results = rb.benchmark(qubits=single_qubits,
                          qc=qc,
                          repeats=20,
                          shots=200,
                          seq_lengths=[1, 5, 10, 15, 20, 25])
rb.plot_results()

### 双量子比特电路
双量子比特电路如图所示

![double-qubit](figures/srb-two-clifford-circuit.png "图3：双量子比特电路。")

方法同上，我们执行代码查看结果

In [ ]:
# 2 qubit SRB
two_qubits = [0,1]
rb2 = StandardRB()
two_qubits_results = rb2.benchmark(qubits=two_qubits,
                          qc=qc,
                          repeats=20,
                          shots=200,
                          seq_lengths=[1, 2, 3, 4, 5, 6])
rb2.plot_results()

运行上述代码，我们即可得到深度 $m$ 与测量结果的拟合曲线（蓝色）以及 $A$, $B$, $f$ 等参数。通过参数我们带入方程计算得到 **EPC**:

$$ {\rm EPC} = \frac{d-1}{d}(1-f),$$

其中 $d=2^N$。我们可以通过下面的代码打印结果进行查看

In [ ]:
print('The fidelity of 1 qubit SRB is :',single_qubit_results['f'])
print('The Error Per Clifford of 1 qubit SRB is:',single_qubit_results['epc'])
print('The fidelity of 2 qubit SRB is :',two_qubits_results['f'])
print('The Error Per Clifford of 2 qubit SRB  is:',two_qubits_results['epc'])

至此我们就完成了 **SRB** 的基本流程，并计算得到了含噪电路的 Clifford 门错误率 **EPC**。
用户可以通过定义不同的噪声，设置不同的参数来进行实验观察结果。
想了解更多随机基准测试及其变种的可以参考这篇论文[3]。


## 参考资料

\[1\] Gottesman, Daniel. "The Heisenberg representation of quantum computers." [arXiv](https://arxiv.org/abs/quant-ph/9807006) preprint quant-ph/9807006 (1998).

\[2\] Selinger, Peter. "Generators and relations for n-qubit Clifford operators." [Logical Methods in Computer Science](https://arxiv.org/abs/1310.6813v4) 11 (2015).

\[3\] Helsen, Jonas, et al. "A general framework for randomized benchmarking." [arXiv](https://arxiv.org/abs/2010.07974) preprint arXiv:2010.07974 (2020).



